# 🤗 x 🦾: Training SmolVLA with LeRobot Notebook

Welcome to the **LeRobot SmolVLA training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `SmolVLA` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `SmolVLA` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


In [ ]:
!wandb login

wandb: Currently logged in as: damin3927 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `colab smolvla ft` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authentica

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

fatal: destination path 'lerobot' already exists and is not an empty directory.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.1.0-py3-none-any.whl size=16366 sha256=15dcbc92fc8fe9542afd3f36129de5af8ab5b343281d499ad864d1e0e716556f
  Stored in directory: /tmp/pip-ephem-wheel-cache-amjkst7m/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
Successfully bu

## Install SmolVLA dependencies

In [ ]:
!cd lerobot && pip install -e ".[smolvla]"

Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 135.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.4 MB/s eta 0:00:00
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.1.0-py3-none-any.whl size=16366 sha256=15dcbc92fc8fe9542afd3f36129de5af8ab5b343281d499ad864d1e0e716556f
  Stored in directory: /tmp/pip-ephem-wheel-cache-8lqg9crn/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=77b6a45120a4350961f6bfb67ce95b732a1098bc1835e1034d9efdf3b57e0974
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f8

## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
!cd lerobot && python lerobot/scripts/train.py \
  --policy.path=lerobot/smolvla_base \
  --policy.repo_id=Damin3927/smolvla_pickplace_ft \
  --dataset.repo_id=Damin3927/so101_pickplace \
  --batch_size=64 \
  --steps=20000 \
  --save_freq=2000 \
  --output_dir=outputs/train/my_smolvla \
  --job_name=smolvla_pickplace_4 \
  --policy.device=cuda \
  --wandb.enable=true \
  --wandb.entity=damin3927-science-tokyo \
  --wandb.project=smolvla_pickplace

config.json: 2.04kB [00:00, 9.32MB/s]
INFO 2025-06-27 12:33:29 ts/train.py:111 {'batch_size': 64,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwa

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli upload Damin3927/so101_pickplace \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model

Start hashing 3 files.
Finished hashing 3 files.
Uploading...: 100% 907M/907M [00:05<00:00, 174MB/s]
https://huggingface.co/Damin3927/so101_pickplace/tree/main/.


In [ ]:
from google.colab import runtime
runtime.unassign()